## Configuração e conexão com o Banco de Dados

In [1]:
import json
from mysql.connector import connection
from tqdm.notebook import tqdm

In [2]:
config = {
  'user': 'admin',
  'password': 'admin',
  'host': 'localhost',
  'database': 'melhorenvio',
  'raise_on_warnings': True
}

In [3]:
cnx = connection.MySQLConnection(**config)

### Funções auxiliares para o banco de dados

In [4]:
def exists(table, id):
    
    cursor = cnx.cursor()
    
    if type(id) == str:
        
        query = "SELECT id FROM {} WHERE id='{}'".format(table, id)        
    
    else:
        
        query = "SELECT id FROM {} WHERE id={}".format(table, id)

    cursor.execute(query)
    
    result = cursor.fetchone()
    
    if result is None:
        return False
    
    return True

In [5]:
def insert(table, columns, values):
    
    cursor = cnx.cursor()
    
    columns = ', '.join(columns)
    values = ", ".join(repr(x) for x in values)
    
    query = "INSERT INTO {} ({}) VALUES ({})".format(table, columns, values)
    
    result = cursor.execute(query)
    
    return cursor.lastrowid

## Criação de uma lista de JSON

In [6]:
# Cria uma lista para armazenar os objetos JSON
logsList = []

# Percorre o arquivo lendo cada linha como um objeto json e adicionando à lista
with open('../raw_data/logs.txt') as f:
    for jsonObj in f:
        log = json.loads(jsonObj)
        logsList.append(log)

In [7]:
exists('request', 0)

False

## Inserção da lista dentro do banco de dados

In [8]:
for log in tqdm(logsList):
    
    # request
    table = 'request'
    request = log[table]
    
    request['headers'] = '["'+ str(request['headers']) + '"]' 
    request['querystring'] = str(request['querystring'])
    
    request['id'] = insert(table, request.keys(), request.values())
    
    # response
    table = 'response'
    response = log[table]
    
    response['headers'] = '["'+ str(response['headers']) + '"]' 
    
    response['id'] = insert(table, response.keys(), response.values())
    
    # authenticated_entity (consumidor)
    table = 'authenticated_entity'
    authenticated_entity = log[table]
    authenticated_entity['consumer_id']['uuid'] = authenticated_entity['consumer_id']['uuid']
    
    if not exists(table, authenticated_entity['consumer_id']['uuid']):
        insert(table, ['id'], [authenticated_entity['consumer_id']['uuid']])
        
    # route
    table = 'route'
    route = log[table]
    route['methods'] = ", ".join(route['methods'])
    route['protocols'] = ", ".join(route['protocols'])
    route['paths'] = ", ".join(route['paths'])
    
        # Remoção do id do serviço pois esse dado está na tabela log 
    route.pop('service', None)
    
    if not exists(table, route['id']):
        insert(table, route.keys(), route.values())
        
    # service
    table = 'service'
    service = log[table]
    
    if not exists(table, service['id']):
        insert(table, service.keys(), service.values())
        
    # latencies
    table = 'latencies'
    latencies = log[table]   
    
    latencies['id'] = insert(table, latencies.keys(), latencies.values())

    # log
    table = 'log'
    columns = [
        'upstream_uri',
        'client_ip',
        'started_at',
        'request_id',
        'response_id',
        'authenticated_entity_id',
        'route_id',
        'service_id',
        'latencies_id'
    ]
    
    values = [
        log['upstream_uri'],
        log['client_ip'],
        log['started_at'],
        request['id'],
        response['id'],
        authenticated_entity['consumer_id']['uuid'],
        route['id'],
        service['id'],
        latencies['id']
    ]
    
    insert(table, columns, values)
    cnx.commit()

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:
cnx.close()

Conferindo se a inserção deu certo:

In [12]:
# Conectando novamente com a base
cnx = connection.MySQLConnection(**config)
cursor = cnx.cursor()
    
query = "SELECT COUNT(*) FROM log;"
cursor.execute(query)
result = cursor.fetchone()

In [13]:
result

(100000,)

Resultado correto, todos os 100mil logs foram armazenados.